<a href="https://colab.research.google.com/github/bcaffo/ds4bme_intro/blob/master/notebooks/notebook5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear separable models

We've now covered two ways to do prediction with a single variable, classification using logistic regression and prediction using a line and least squares. What if we have several predictiors? 

In both the logistic and linear regression models, we had a linear predictor, specifically, 
$$
\eta_i = \beta_0 + \beta_1 x_i.
$$
In the continuous case, we were modeling the expected value of the outcomes as linear. In the binary case, we were assuming that the naturual logarithm of the odds of a 1 outcome was linear. 

To estimate the unknown parameters, $\beta_0$ and $\beta_1$ we minimized
$$
\sum_{i=1}^n || y_i - \eta_i||^2 
$$
in the linear case and 
$$
-\sum_{i=1}^n \left[
  Y_i \eta_i + \log\left\{\frac{1}{1 + e^\eta_i} \right\} \right].
$$
in the binary outcome case (where, recall, $\eta_i$ depends on the parameters). 

We can easily extend these models to multiple predictors by assuming that the impact of the multiple predictors is linear and separable. That is,

$$
\eta_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \ldots \beta_{p-1} x_{p-1,i}
$$

If we think about this as vectors and matrices, we obtain

$$
\eta = X \beta
$$
where $\eta$ is an $n \times 1$ vector, $X$ is an $n \times p$ matrix with $i,j$ entry $x_{ij}$ and $\beta$ is a $p\times 1$ vector with entries $\beta_j$. 


Let's look at the voxel-level data that we've been working with. First let's load the data.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import statsmodels.formula.api as smf
import statsmodels as sm

## this sets some style parameters
sns.set()

## Download in the data if it's not already there
! if [ ! -e oasis.csv ]; \
then wget https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv; \
fi;

## Read in the data and display a few rows
dat = pd.read_csv("oasis.csv")

--2019-09-23 18:19:04--  https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22274 (22K) [text/plain]
Saving to: ‘oasis.csv’

oasis.csv           100%[===================>]  21.75K  --.-KB/s    in 0.01s   

2019-09-23 18:19:04 (1.77 MB/s) - ‘oasis.csv’ saved [22274/22274]



Let's first try to fit the proton density data from the other imaging data. I'm going to use the `statsmodels` version of linear models since it has a nice format for dataframes.

In [0]:
trainFraction = .75

sample = np.random.uniform(size = 100) < trainFraction
trainingDat = dat[sample]
testingDat = dat[~sample]

In [4]:
results = smf.ols('PD ~ FLAIR + T1 + T2  + FLAIR_10 + T1_10 + T2_10 + FLAIR_20', data = trainingDat).fit()
print(results.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.732   
Dependent Variable: PD               AIC:                71.0012 
Date:               2019-09-23 18:19 BIC:                89.1026 
No. Observations:   71               Log-Likelihood:     -27.501 
Df Model:           7                F-statistic:        28.34   
Df Residuals:       63               Prob (F-statistic): 3.38e-17
R-squared:          0.759            Scale:              0.14318 
------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025  0.975]
------------------------------------------------------------------
Intercept       0.4725    0.1517   3.1153  0.0028   0.1694  0.7756
FLAIR          -0.1419    0.0997  -1.4234  0.1596  -0.3412  0.0573
T1             -0.1510    0.0961  -1.5710  0.1212  -0.3430  0.0411
T2              0.7111    0.0948   7.5020  0.0000   0.5217  0.9005
FLAIR_10       -0.09

In [8]:
x = dat[['FLAIR','T1', 'T2', 'FLAIR_10', 'T1_10', 'T2_10', 'FLAIR_20']]
y = dat[['GOLD_Lesions']]
## Add the intercept column
x = sm.tools.add_constant(x)

xtraining = x[sample]
xtesting = x[~sample]
ytraining = y[sample]
ytesting = y[~sample]


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [12]:
fit = sm.discrete.discrete_model.Logit(ytraining, xtraining).fit()

Optimization terminated successfully.
         Current function value: 0.241546
         Iterations 8


In [13]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           GOLD_Lesions   No. Observations:                   71
Model:                          Logit   Df Residuals:                       63
Method:                           MLE   Df Model:                            7
Date:                Mon, 23 Sep 2019   Pseudo R-squ.:                  0.6515
Time:                        18:22:05   Log-Likelihood:                -17.150
converged:                       True   LL-Null:                       -49.206
Covariance Type:            nonrobust   LLR p-value:                 2.266e-11
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7783      1.985     -2.407      0.016      -8.668      -0.888
FLAIR          2.6983      1.353      1.994      0.046       0.046       5.351
T1             3.0484      1.306      2.334      0.020       0.489       5.608
T2             2.1443      1.196      1.793      0.073      -0.199       4.488
FLAIR_10       4.9421      4.256      1.161      0.246      -3.400      13.284
T1_10         -0.3192      2.331     -0.137      0.891      -4.888       4.250
T2_10         -4.9192      4.014     -1.226      0.220     -12.786       2.947
FLAIR_20     -16.6939      9.496     -1.758      0.079     -35.306       1.918
==============================================================================
"""

Now let's evaluate our prediction. Here, we're not going to classify as 0 or 1, but rather estimate the prediction. Note, we then would need to pick a threshold to have a classifier. We could use .5 as our threshold. However, it's often the case that we don't necessarily want to threshold at specifically that level. A solution for evalution is to plot how the sensitivity and specificity change by the threshold. 

In other words, consider the triplets
$$
(t, sens(t), spec(t))
$$
where $t$ is the threshold, `sens(t)` is the sensitivity at threshold $t$, `spec(t)` is the specificity at threshold `t`. 

Necessarily, the sensitivity and specificity 



In [0]:
phatTesting = fit.predict(xtesting)

## Aside different python packages

So far we've explored several plotting libraries including: default pandas methods, matplotlib, seaborn and plotly. We've also looked at several fitting libraries including to some extent numpy, but especially scikitlearn and statsmodels. What's the difference? Well, these packages are all mantained by different people and have different features and goals. For example, scikitlearn is more expansive than statsmodels, but statsmodels functions more like one is used to with statistical output. Matplotlib is very expansive, but seaborn has nicer default options and is a little easier. So, when doing data science with python, one has to get used to trying out a few packages, weighing the cost and benefits of each, and picking one. 

'statsmodels', what we're using above, has multiple methods for fitting binary models including: `sm.Logit`, `smf.logit`, `BinaryModel` and `glm`. Here I'm just going to use `Logit` which does not use the formula syntax of `logit`. Note, by default, this does not add an intercept this way. So, I'm adding a column of ones, which adds an intercept.

Consider the following which uses the formula API


In [17]:
results = smf.logit(formula = 'GOLD_Lesions ~ FLAIR + T1 + T2 + FLAIR_10 + T1_10 + T2_10 + FLAIR_20', data = trainingDat).fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.241546
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           GOLD_Lesions   No. Observations:                   71
Model:                          Logit   Df Residuals:                       63
Method:                           MLE   Df Model:                            7
Date:                Mon, 23 Sep 2019   Pseudo R-squ.:                  0.6515
Time:                        18:23:47   Log-Likelihood:                -17.150
converged:                       True   LL-Null:                       -49.206
Covariance Type:            nonrobust   LLR p-value:                 2.266e-11
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.7783      1.985     -2.407      0.016      -8.668      -0.888
FLAIR          2.6983      1.353      1.994      0.046       0.046       5.351
T1             3.0484      1.306      2.334      0.020       0.489       5.608
T2             2.1443      1.196      1.793      0.073      -0.199       4.488
FLAIR_10       4.9421      4.256      1.161      0.246      -3.400      13.284
T1_10         -0.3192      2.331     -0.137      0.891      -4.888       4.250
T2_10         -4.9192      4.014     -1.226      0.220     -12.786       2.947
FLAIR_20     -16.6939      9.496     -1.758      0.079     -35.306       1.918
==============================================================================
"""